In [134]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from python_graphql_client import GraphqlClient

In [135]:
# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint="https://api.thegraph.com/subgraphs/name/nounsdao/nouns-subgraph")

In [136]:
# Create the query string and variables required for the request.
query = """
    {
  nouns(first: 1000, orderBy: id) {
    id,
    owner,
    votes {
        proposal {
            id,
            proposer{id},
            proposalThreshold,
            quorumVotes,
            forVotes,
            againstVotes,
            abstainVotes,
            description,
            status,
            votes{id,support,votes,reason}

        },
        supportDetailed
    }
  }
}
"""
# Synchronous request
data = client.execute(query=query)

# For noun in data["data"]["nouns"] get each ['votes'] ['proposal'] and add them all to a list
proposals = []
for noun in data["data"]["nouns"]:
    for vote in noun["votes"]:
        proposals.append(vote["proposal"])


In [137]:
proposals_df = pd.DataFrame(proposals)
# Replace all "\n" with " " in the description column
proposals_df["description"] = proposals_df["description"].str.replace("\n", " ")
# each row of ['proposer'] is a dictionary {'id':'...'}, so we need to just keep the 'id'
proposals_df["proposer"] = proposals_df["proposer"].apply(lambda x: x["id"])


In [138]:
# change default row and column display limit to none
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [139]:
proposals_df = proposals_df.loc[proposals_df.astype(str).drop_duplicates().index]
proposals_df.shape


(100, 10)

In [140]:
# sort by id    
proposals_df = proposals_df.sort_values(by=['id'])
proposals_df.tail()

,id,proposer,proposalThreshold,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,votes
263,94,0x8321795bdace7428d4fadfa30c2ee8e727e491e5,0,33,52,33,0,# Nouns cc0 Space Image Bank # TLDR: Space i...,EXECUTED,[{'id': '0x021edd67d43b365a6401a5ee704aa6f264f...
74,95,0x36a5bc205df1ed65c86301022cfc343a6ce546ff,0,33,139,0,0,"# Nouns @ Art Basel (Basel, Switzerland) # tl...",EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...
75,97,0x91dccaa260cc4616e1a6e6b693db7207c5e42937,0,34,80,4,6,# MOOØNBEANS: Nouns Baked Beans **TL;DR** MO...,EXECUTED,[{'id': '0x2573c60a6d127755aa2dc85e342f7da2378...
228,98,0x36a5bc205df1ed65c86301022cfc343a6ce546ff,0,34,42,0,6,# NOUNS IN RIO - Nouns to be a part of the 1st...,EXECUTED,[{'id': '0x6f9e3976fa3b5b22761fe5d635e1f0d9d9a...
229,99,0x75ee6eb3d8dacf41ee2e5307090b197d3e1cca6e,0,35,77,2,0,"# Sailing PR campaign, Korea Blockchain Week 2...",CANCELLED,[{'id': '0x008c84421da5527f462886cec43d2717b68...


In [141]:
# Create a new dataframe then loop through all rows and for each row loop through all 'votes'
# Each vote that has a 'reason' we will append a new row to the new dataframe
# The new dataframe will have the columns 'proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason'

reasons_df = pd.DataFrame()
for index, row in proposals_df.iterrows():
    for vote in row["votes"]:
        if vote["reason"] is not None:
            if vote['support']:
                # append using concat to add a new row to the dataframe into the correct columns saying "I support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# The proposal brought forth to the Nouns decentralized autonomous organization for funding by the shared treasurey is: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I support this proposal, here is why: " + vote["reason"] + "## END"]])], ignore_index=True)
            else:
                # append using concat to add a new row to the dataframe into the correct columns saying "I do not support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# The proposal brought forth to the Nouns decentralized autonomous organization for funding by the shared treasurey is: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I do not support this proposal, here is why: " + vote["reason"] + "## END"]])], ignore_index=True)

# rename the columns
reasons_df.columns = ['proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason']

In [142]:
# print the shape of the new dataframe
print(reasons_df.shape)
# print all the unique proposal ids sorted low to high
print(reasons_df["proposal_id"].sort_values().unique().tolist())
# print how many unique proposal ids there are
print(len(reasons_df["proposal_id"].sort_values().unique().tolist()))

(312, 11)
['100', '101', '102', '103', '104', '105', '106', '44', '47', '49', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '67', '68', '69', '70', '71', '72', '73', '74', '75', '77', '78', '81', '82', '83', '84', '85', '86', '87', '89', '90', '91', '92', '93', '94', '95', '97', '98', '99']
51


In [143]:
reasons_df.head()

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
0,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0x008c84421da5527f462886cec43d2717b686a7e4-100,True,5,"## I support this proposal, here is why: The d..."
1,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0x203cc9b81c53fd649bd575e0e19925dd1a2a54da-100,False,1,"## I do not support this proposal, here is why..."
2,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xa86882277e69fbf0a51805cdc8b0a3a113079e63-100,True,2,"## I support this proposal, here is why: The n..."
3,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xbd7dbab9aeb52d6c8d0e80fcebde3af4cc86204a-100,True,6,"## I support this proposal, here is why: ⌐◨-◨ ..."
4,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-100,True,12,"## I support this proposal, here is why: ⌐◨-◨ ..."


In [144]:
# print length of the longest "reason"
print(max([len(x) for x in reasons_df["reason"]]))

8336


In [145]:
# print the full "reason" of any row with a duplicate reason
reasons_df.loc[reasons_df.duplicated(subset="reason", keep=False)]

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
15,102,35,68,1,2,# The proposal brought forth to the Nouns dece...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-102,True,12,"## I support this proposal, here is why: ⌐◨-◨#..."
19,103,36,114,1,0,# The proposal brought forth to the Nouns dece...,QUEUED,0x73222225044ce0fc672ff9b6f9730d0a0421382c-103,True,1,"## I support this proposal, here is why: ⌐◨-◨\..."
20,103,36,114,1,0,# The proposal brought forth to the Nouns dece...,QUEUED,0x88f9e324801320a3fc22c8d045a98ad32a490d8e-103,True,3,"## I support this proposal, here is why: ⌐◨-◨#..."
23,103,36,114,1,0,# The proposal brought forth to the Nouns dece...,QUEUED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-103,True,12,"## I support this proposal, here is why: ⌐◨-◨#..."
29,104,36,68,29,0,# The proposal brought forth to the Nouns dece...,QUEUED,0x88f9e324801320a3fc22c8d045a98ad32a490d8e-104,True,3,"## I support this proposal, here is why: ⌐◨-◨#..."
33,104,36,68,29,0,# The proposal brought forth to the Nouns dece...,QUEUED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-104,True,12,"## I support this proposal, here is why: ⌐◨-◨#..."
38,105,36,74,0,0,# The proposal brought forth to the Nouns dece...,ACTIVE,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-105,True,12,"## I support this proposal, here is why: ⌐◨-◨#..."
53,56,27,39,1,2,# The proposal brought forth to the Nouns dece...,EXECUTED,0x8323f1c687f7e2296ec71ee3549a7430ea7ec730-56,True,1,"## I support this proposal, here is why: ⌐◨-◨#..."
57,57,27,48,8,16,# The proposal brought forth to the Nouns dece...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-57,True,26,"## I support this proposal, here is why: ⌐◨-◨#..."
89,62,27,71,0,0,# The proposal brought forth to the Nouns dece...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-62,True,25,"## I support this proposal, here is why: ⌐◨-◨#..."


In [146]:
# we see some artifacts in the text so we will make a preprocessing function to clean up the text
def clean_text(text):
    """
    There are a few artifacts to clean namely backslash n and backslash r 
    To approach this we can remove all mention of them and then replace them with a space
    We'll then have to remove all of the duplicate spaces
    """
    text = text.replace(r"\n", " ")
    text = text.replace(r"\u", " ")
    text = text.replace(r"\t", " ")

    # remove all duplicate spaces
    text = " ".join(text.split())
    return text
    

In [147]:
# apply the clean_text function to the "reason" column
reasons_df["reason"] = reasons_df["reason"].apply(clean_text)

In [148]:
# rename 'description' to 'prompt' and 'reason' to 'completion'
reasons_df.rename(columns={"description": "prompt", "reason": "completion"}, inplace=True)

In [149]:
# remove the row with proposal id '100' to use as a test, first save the row
test_rows = reasons_df.loc[reasons_df["proposal_id"] == "100"]
# remove the row with proposal id '100'
reasons_df = reasons_df.loc[reasons_df["proposal_id"] != "100"]


In [150]:
test_rows

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,prompt,status,voter id,voter support,votes,completion
0,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0x008c84421da5527f462886cec43d2717b686a7e4-100,True,5,"## I support this proposal, here is why: The d..."
1,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0x203cc9b81c53fd649bd575e0e19925dd1a2a54da-100,False,1,"## I do not support this proposal, here is why..."
2,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xa86882277e69fbf0a51805cdc8b0a3a113079e63-100,True,2,"## I support this proposal, here is why: The n..."
3,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xbd7dbab9aeb52d6c8d0e80fcebde3af4cc86204a-100,True,6,"## I support this proposal, here is why: ⌐◨-◨ ..."
4,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-100,True,12,"## I support this proposal, here is why: ⌐◨-◨ ..."
5,100,35,43,1,1,# The proposal brought forth to the Nouns dece...,EXECUTED,0xf6c625851d48fe26b50d96b821a87efc7e35c222-100,True,1,"## I support this proposal, here is why: I am ..."


In [151]:
# save the dataframe to a csv file
reasons_df.to_csv("nouns_vote_reasons.csv", index=False)

In [167]:
test_prompt = """
Free Glasses For Kids v4

Description
By Jawshua Fisher & Fitz Frames

tl;dr:
After completing a succesful pilot we can expand the Nouns X Fitz Frames Prescription Subscription to 3,000 Kids for $300,000

In addition we can pilot a Nounish Vision Van Program with Sergul Ezurum, MD, founder of Sight For All United who will spearhead this initiative in parternship with Vision To Learn. Together we will create a high tech, Nounish Vision Van Program and execute a years worth of exams covering 3-4,000 kids for $250,000

*there might be some overlap between NxFF Subscription Prescriptions and Sight For All United's program but they are not inclusive of each other.

Recap: Prop 12: Nouns x Fitz Frames Free Glasses For Kids v2:
Nouns x Fitz Free Glasses For Kids v2 was a huge success. We were able to visit 3 schools, complete 355 eye exams and dispense subscriptions for 285 kids in need, delivering a total of 391 custom fitted 3D printed Fitz Frames glasses to date.

We spent a significant effort developing Noun Frames which inspired other builders and led the way for FUN Frames and Fashion Frames to move into development.

Highlights & Updates: https://discord.com/channels/849745721544146955/903032675043016746/920205659675439116

Nouns Frames Pic|326x500, 50%

https://vimeo.com/708413500

Nouns x Fitz Frames FGFK v4:
We want to do this again, but this time let’s do more!

Continuing our partnership with Fitz Frames & partnering with various vision screening and testing providers we will deliver 3,000 Nouns X Fitz Frames Prescription Subscriptions.

The Nouns X Fitz Frames Prescription Subscription, which provides free glasses for kids if they lose them, break them, grow out of them or just desires a new pair because their style changed, will be expanded to 3K kids. Using a propriatery in-app measurement and delivery system Fitz Frames removes the need for in-person fittings and shapings that require specialized equipment and gets the perfect custom fit 3D printed glasses for kids individual face shape and personal style. Our goal is to keep kids in glasses that fit and they're proud to wear, even as they change throughout the year.

Expanding the scale of this program will get us one step closer to our moon shot of creating a system where individual doctors around the world can plug in to and get free glasses for kids in need through the Nouns X Fitz Frames Prescription Subscription program.

As a pivot from a Nationwide Sponsorship for Vision To Learn we have the opportunity to work hand in hand with Sight For All United on launching a new Vision Van Program complete with a brand new van that we can collaborate together on to make it as Nounish as possible. This will allow us to peek under the hood, literally, at how Vision To Learn works and the impact these programs deliver to communities where they are available.

Nouns X Fitz Prescription Subscription - $300,000
Nounish Vision Van - $250,000

PHASES OF EXECUTION
PHASE 1:
150K - Fitz Frames
150K - Nounish Vision Van Program / Sergul

PHASE 2: Fund second 50% of proposed allocation
150K - After delivery of 1K Prescription Subscriptions
100K - Nounish Vision Van Program - Paid upon delivery of the Van

Send $575K (~550ETH) equivalent of ETH to Fitz Frames Gnosis from v2. Added some cushion for volatility during prop window. Will work with Seneca to convert and then pay front end and send ETH balance back to the DAO. Will pay backend upon reaching benchmarks.

Gnosis Address: 0xC7C696c74490b1ae1b705d7A2a6058B5591DE641

Additional Opportunities
Press Opportunites
Between testing, fullfillment and event there are many opportunities for great press.
Fitz Frames our screening partners have great press contacts that we can talk with.

Branding on Fitz Frame Glasses
For the Nouns X Fitz Frams Prescription Subscription glasses we can include Nouns logo on the temples or even a Noun itself.
Could possibly upgrade to letting kids pick the noun they want.

Nounish Vision Van - Branding on Mobile Vision Clinic
We can work hand in hand on creating a wonderful Nounish Vision Van with Sight For All United.

Nounish Gift Packs for Kids
Nouns Gift Packs that can include mutually approved items such as FUN Frames, stickers, coloring books, keychain, and more.
Bigshot Toy Works has commmitted to helping find the right package and offered fullfillment. Will deterimine on a case by case basis which partners can deliver which items.


About Fitz:
Fitz Frames: https://www.fitzframes.com/
Fitz for Good - how Fitz has worked with VTL to provide glasses for kids in the past:
https://vimeo.com/377468371
How Fitz has worked with Healthcare community to provide protective prescription eyewear to drs and nurses across the country - https://news.yahoo.com/local-hero-nyc-glasses-maker-171104609.html
Fitz in the Press - How we came to be and why we matter to kids - https://abcnews.go.com/Business/startup-offers-affordable-eyeglasses-kids-flatter-faces/story?id=65014415

About Vision 2 Learn:
https://visiontolearn.org/about/from-the-founder/

Proposed Transactions
0xc7c696c74490b1ae1b705d7a2a6058b5591de641.transfer(
 550.0 ETH
)


"""

In [168]:
# prompt = the prompt from the first row of test_rows with the suffix `\n\n###\n\n
#prompt = test_rows["prompt"].values[0] + "\n\n###\n\n"

# prompt = test_prompt with the suffix `\n\n###\n\n`
prompt = test_prompt + "\n\n###\n\n ## I"

In [182]:
import openai
max_tokens = 300

"""
call openai completion and receinve a json response that looks like:

<OpenAIObject text_completion id=cmpl-5RtlbfOAMGfFjskajxT9llnxNnsIN at 0x181e87f92b0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "I support this proposal: everyone in this room loves a good contest + Noun O' Clock + great finesse\nmore desert raven\nits like a mushroom\n\ud83d\udc25 ENDORSEMENT\none more one\nleather boy\nkleet cheez mac\n\ud83d\udc10 ENDORSEMENT\nMORE ENDORSEMENT\n\ud83c\udf10 ENDORSEMENT\ni'll be here\nendorsing: ser noadz ENDORSEMENT\nmore noun I ENDORSEMENT\n\ud83d\udc2e ENDORSEMENT\n\ud83d\udc30 ENDORSEMENT\n\u26a0 ENDORSEMENT\n\ud83d\udc30 ENDORSEMENT\n\ud83d\udc1c ENDORSEMENT\n\ud83d\udc30 endorment \u2326 ENDORSEMENT\n\ud83c\udfa5 END"
    }
  ],
  "created": 1657330907,
  "id": "cmpl-5RtlbfOAMGfFjskajxT9llnxNnsIN",
  "model": "ada:ft-personal-2022-06-29-00-28-59",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 150,
    "prompt_tokens": 1893,
    "total_tokens": 2043
  }
}

"""
openai_response = openai.Completion.create(
    model="curie:ft-personal-2022-07-09-02-36-32",
    prompt=prompt, max_tokens = max_tokens)


In [183]:
# take out only the text from the response
openai_response_text = openai_response.choices[0].text
# add a prefix of "I"
openai_response_text = "I " + openai_response_text

# if the response includes " ## END" stop the resopnse at "## END
if "## END" in openai_response_text:
    openai_response_text = openai_response_text.split("## END")[0]

# print out the response
print(openai_response_text)

I  support this prop and this transaction gets the attention of the community which is imortant for validating Nounish within the community.


In [133]:
openai_response.choices[0].text

' support this proposal, here is why: good gm, secure returns, and eth to a'